In [ ]:
import requests, base64, json, urllib, time, re
import pandas as pd

cleaned_data = 'xxxx'
tdata = pd.DataFrame.from_csv(cleaned_data, sep='\t')
sample_count = tdata['tweet_id'].count()

In [ ]:
# now to get the bearer token
uenc_con_key = consumer_key # right now encoding doesn't change format
uenc_con_secret = consumer_secret # right now encoding doesn't change format
concat_string = uenc_con_key + ':' + uenc_con_secret
b64_concat_string = base64.b64encode(concat_string)
body = "grant_type=client_credentials"
headers = {}
headers['Authorization'] = 'Basic ' + b64_concat_string
headers['Content-Type'] = 'application/x-www-form-urlencoded;charset=UTF-8'
r = requests.post(oauth_url, data=body, headers=headers)

In [ ]:
# now prep for the actual batch call 
access_token = r.json()['access_token']
b64_bearer = base64.b64encode(access_token)
headers = {}
headers['Authorization'] = 'Bearer ' + access_token

In [1]:
# now that we have the headers, we set up some basic variables
tweet_csv_prefix = 'tweets_lang_'
csv_suffix = '.csv'

In [ ]:
tweet_url = 'https://api.twitter.com/1.1/statuses/lookup.json?id='
tweet_number = 99
idx = 0
delim = ','
sleep_time = 10 # in seconds

while idx < sample_count: 
    
    # batch call to get 99 tweets
    tweet_full = tweet_url + str(tweet_number) + delim
    idx_end = idx + tweet_number
    ids = ""
    
    # concat tweet ids 
    for i in tdata.iloc[idx:idx_end]['tweet_id']: 
        ids += str(i) + delim
        
    tweet_full = tweet_full + ids[:-1] # remove last comma 
    tweet_full = tweet_full[:-1]
    
    # make requests for 99 tweets at a time
    r = requests.get(tweet_full, headers=headers)
    
    # if the status is not successful, 
    # continue without changing idx
    # so we can re-try the index 
    if (r.status_code != 200):
        print "error: " + r.text
        time.sleep(60)
        continue
    tweet = r.text
    jtweet = json.loads(tweet)
    lang_list = []
    text_list = []
    # store found tweets and corresponding langs
    for t in jtweet: 
        if t['text'] is not None and len(t['text']) > 0: 
            text = t['text']
            text = re.sub('\n', '', text)
            lang_loc = tdata[tdata['tweet_id'] == t['id']]
            if not lang_loc.empty:
                lang = lang_loc['language'].values[0]
                lang_list.append(lang)
                text_list.append(text)
        else: 
            print t['id'] + " not found!"
    
    # write to data structure
    smp = []
    smp = zip(text_list, lang_list)
    
    # store in temporary csv file 
    idx += tweet_number
    print "End idx: ", idx # csv file tracks last index we stopped at 
    csv_name = tweet_csv_prefix + str(idx) + csv_suffix
    df = pd.DataFrame(data=smp, columns=['tweet', 'lang'])
    df.to_csv(csv_name, index=False, header=False, sep='\t', encoding='utf-8')
    time.sleep(sleep_time)